In [ ]:
import subprocess
import numpy as np
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
from astropy.table import Table
from astropy.io import fits
from astropy import wcs
from glob import glob
import shlex
from regions import read_ds9, write_ds9
import numpy.ma as ma
        
# parallel processor et al.
from utilities import parallel_process, check_exe, system_call
from utilities import get_immediate_subdirectories

import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)
from astropy import log
log.setLevel('WARN')

In [ ]:
def write_single_xspec(name, outpath, z):
    
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        return

    # now we have to loop through the detections
    for j in detects['INDEX']:
        if not os.path.isfile(
                f'{outpath}/{name}/spec_files/{j}_addascaspec.log'):
            continue
        
        with open(f'{outpath}/{name}/spec_files/{j}_addascaspec.log', 'r') as logfile:
            lines = logfile.readlines()
        
        specset = {}
        counter = 1
        
        specset[counter] = {}
        for line in lines:
            if 'Spec:' in line:
                if counter > 1:
                    specset[counter] = {}
                specset[counter]['Spec'] = line.split(':')[-1]
            elif 'Bgd :' in line:
                specset[counter]['Bkg'] = line.split(':')[-1]
            elif 'ARF :' in line:
                specset[counter]['ARF'] = line.split(':')[-1]
            elif 'RMF :' in line:
                specset[counter]['RMF'] = line.split(':')[-1]
                counter += 1

        with open(f'{outpath}/{name}/spec_files/{j}_singles_xspec.in',
                  'w') as script:

            for i in range(1, counter):
                
                # here is the text we are going to write
                text = [
                    f"data {outpath}/{name}/spec_files/{specset[i]['Spec']}",
                    f"resp {specset[i]['RMF']}",
                    f"arf {outpath}/{name}/spec_files/{specset[i]['ARF']}",
                    f"back {outpath}/{name}/spec_files/{specset[i]['Bkg']}",
                    "ignore bad\n",
                    "ignore **-0.5,2.0-**\n",
                    "show rates\n"
                ]

                #Write it to the script
        
                for line in text:
                    script.write(line)      
                
            script.write('Exit')    
         
    return 

In [ ]:
def run_single_xspec(name, outpath, z):
    
    if not z >= 0.1 or z < 0:
        return
    
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        return
#         raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')

    results = {}
    results['field'] = name
    
    # now we have to loop through the detections
    for j in detects['INDEX']:
        if not os.path.isfile(f'{outpath}/{name}/spec_files/{j}_singles_xspec.in'):
            continue
        else:
            source_xspec_in = f'{outpath}/{name}/spec_files/{j}_singles_xspec.in'
            
        # build the command and call it
        cmd = f"xspec < {source_xspec_in}"
        stdout, stderr = system_call(cmd, shlexify=False)
                
        # log
        log_file = f'{outpath}/{name}/spec_files/{j}_singles_xspec.log'
        with open(log_file, 'w') as f:
            f.writelines(stdout)
         
        
        ##################
        #### Parse the log file to get the values we want ####
        ##################
        
        # individual source result
        results[j] = {}    
        
        # janky way of parsing the file
        count = 1
        
        outlines = stdout.split('\n')
        for line in outlines:
            
            if 'XSPEC12>data' in line:
                obs = line.split('/')[-1].split('_')[0]
                results[j][obs] = {}
                
            elif "Net count rate" in line:
                if count > 2:
                    cnt_rate = {}
                    cnt_rate['Rate'] = line.split(' ')[7]
                    cnt_rate['Err'] = line.split(' ')[9]
                    results[j][obs] = cnt_rate
                    count = 1
                else:
                    count += 1

            else:
                pass

            
        ### add the total result to the end
        with open(f'{outpath}/{name}/spec_files/{j}_xspec.log') as total_file:
            outlines = total_file.readlines()
            
        for line in outlines:

            if "Net count rate" in line:
                cnt_rate = {}
                cnt_rate['Rate'] = line.split(' ')[7]
                cnt_rate['Err'] = line.split(' ')[9]
            else:
                pass
    
        results[j]['total'] = cnt_rate    
            
    return results

In [ ]:
# get file data
data = load_PSZcatalog()
data = data.sort_index(axis=1)

outpath = './data_full'

arr = [{'name':n.replace(' ', '_'), 'outpath':outpath, 'z':z} for n, z in zip(data['NAME'], data['REDSHIFT'])]
parallel_process(arr, write_single_xspec, use_kwargs=True, n_jobs=6)
results = parallel_process(arr, run_single_xspec, use_kwargs=True, n_jobs=6)

In [ ]:
# outpath ='./data_full'

# name = 'PSZ2_G111.75+70.37'

# write_single_xspec(name, outpath)
# results = run_single_xspec(name, outpath)

In [ ]:
results

In [ ]:
cleanedresults = [x for x in results if x is not None]

In [ ]:
r = cleanedresults[0]

In [ ]:
r

In [ ]:
list(r[indx[0]].keys())[:-1]

In [ ]:
r[indx[0]]['00090909001']

In [ ]:
sigma = []
cnt = 1
for r in cleanedresults:
    try:
        sources = list(r.keys())[1:]
    except AttributeError: # it's not a dict
        continue
    if len(sources) < 1:
        continue
    for indx in sources:
        obs_id = list(r[indx].keys())[:-1]
        if not len(obs_id) > 1:
            continue
        for obs in obs_id:
            try:
                rate = float(r[indx][obs]['Rate'])
                err = float(r[indx][obs]['Err'])
                total = float(r[indx]['total']['Rate'])
            except KeyError: # no flux or cnt_rate recorded
                continue
            if err == 0:
                continue
            s = (rate - total) / err
            if abs(s) > 3:
                print(r['field'], indx, obs, s)
#             if cnt % 10 == 0:
#                 print(r['field'], indx, obs, s)
            cnt += 1
            sigma.append((rate - total)/ err)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
plt.hist(sigma, bins=25, range=[-10,10])
plt.xlabel('"Sigma"')

In [ ]:
plt.savefig('count_rate_sigma.png')

In [ ]:
np.mean(sigma)

In [ ]:
min(sigma)

In [ ]:
max(sigma)